In [4]:
import hashlib
import json
import time
from typing import List, Dict, Optional

In [12]:
"""
Кошелек для хранения ключей и создания транзакций
"""
class Wallet:
    def __init__(self):
        self.private_key = hashlib.sha256(str(time.time()).encode()).hexdigest()
        self.public_key = hashlib.sha256(self.private_key.encode()).hexdigest()
    
    def create_transaction(self, to_address, amount):
        return Transaction(self.public_key, to_address, amount)

In [13]:
"""
Транзакция биткоина
"""
class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.timestamp = time.time()
        self.signature = self._generate_signature()
    
    def _generate_signature(self):
        data = f"{self.from_address}{self.to_address}{self.amount}{self.timestamp}"
        return hashlib.sha256(data.encode()).hexdigest()
    
    def to_dict(self):
        return {
            'from_address': self.from_address,
            'to_address': self.to_address,
            'amount': self.amount,
            'timestamp': self.timestamp,
            'signature': self.signature
        }

In [14]:
"""
Блок в блокчейне
"""
class Block:
    def __init__(self, transactions, previous_hash):
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        block_data = {
            'timestamp': self.timestamp,
            'transactions': [tx.to_dict() for tx in self.transactions],
            'previous_hash': self.previous_hash,
            'nonce': self.nonce
        }
        return hashlib.sha256(json.dumps(block_data, sort_keys=True).encode()).hexdigest()
    
    def mine_block(self, difficulty):
        target = '0' * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
    
    def to_dict(self):
        return {
            'hash': self.hash,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'nonce': self.nonce,
            'transactions': [tx.to_dict() for tx in self.transactions]
        }


In [18]:
"""
Блокчейн
"""
class Blockchain:
    """Блокчейн"""
    def __init__(self):
        self.chain: List[Block] = [self.create_genesis_block()]
        self.difficulty = 2
        self.pending_transactions: List[Transaction] = []
        self.mining_reward = 100
    
    def create_genesis_block(self):
        return Block([], "0")
    
    def get_latest_block(self):
        return self.chain[-1]
    
    def add_transaction(self, transaction):
        self.pending_transactions.append(transaction)
    
    def mine_pending_transactions(self, mining_reward_address):
        block = Block(self.pending_transactions, self.get_latest_block().hash)
        block.mine_block(self.difficulty)
        
        self.chain.append(block)
        
        self.pending_transactions = [
            Transaction(None, mining_reward_address, self.mining_reward)
        ]
    
    def get_balance(self, address):
        balance = 0
        
        for block in self.chain:
            for transaction in block.transactions:
                if transaction.from_address == address:
                    balance -= transaction.amount
                if transaction.to_address == address:
                    balance += transaction.amount
        
        return balance
    
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            
            if current_block.hash != current_block.calculate_hash():
                return False
            
            if current_block.previous_hash != previous_block.hash:
                return False
        
        return True
    
    def print_chain(self):
        for i, block in enumerate(self.chain):
            print(f"Block {i}:")
            print(f"Hash: {block.hash}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Nonce: {block.nonce}")
            print("Transactions:")
            for tx in block.transactions:
                print(f"  {tx.from_address} -> {tx.to_address}: {tx.amount}")
            print("-" * 50)

In [17]:
bitcoin = Blockchain()
alice = Wallet()
bob = Wallet()
miner = Wallet()
    
print("Создаем транзакции...")
# Алиса отправляет Бобу 50 биткоинов
tx1 = alice.create_transaction(bob.public_key, 50)
bitcoin.add_transaction(tx1)
    
# Боб отправляет Алисе 25 биткоинов
tx2 = bob.create_transaction(alice.public_key, 25)
bitcoin.add_transaction(tx2)
    
print("Майним блок...")
bitcoin.mine_pending_transactions(miner.public_key)
    
print("Создаем еще транзакции...")
# Еще одна транзакция
tx3 = alice.create_transaction(bob.public_key, 10)
bitcoin.add_transaction(tx3)
    
print("Майним второй блок...")
bitcoin.mine_pending_transactions(miner.public_key)
    
print("\nИнформация о блокчейне:")
bitcoin.print_chain()
    
print("\nБалансы:")
print(f"Алиса: {bitcoin.get_balance(alice.public_key)}")
print(f"Боб: {bitcoin.get_balance(bob.public_key)}")
print(f"Майнер: {bitcoin.get_balance(miner.public_key)}")
    
print(f"\nБлокчейн валиден: {bitcoin.is_chain_valid()}")

Создаем транзакции...
Майним блок...
Создаем еще транзакции...
Майним второй блок...

Информация о блокчейне:
Block 0:
Hash: 78dc5fb33bfa6a047a305fb8677eae8aedfeab1256d708faf0c2a3930a0569d0
Previous Hash: 0
Nonce: 0
Transactions:
--------------------------------------------------
Block 1:
Hash: 0034cf1d48a0ad1a32eca65314f3a8609c67666e0ebc516316d9c1f4a0ac7687
Previous Hash: 78dc5fb33bfa6a047a305fb8677eae8aedfeab1256d708faf0c2a3930a0569d0
Nonce: 70
Transactions:
  d62cbf84ddeab29b7c3c78c09a7e7d27debc68018c15f4525bc42c7ffd74e98e -> d62cbf84ddeab29b7c3c78c09a7e7d27debc68018c15f4525bc42c7ffd74e98e: 50
  d62cbf84ddeab29b7c3c78c09a7e7d27debc68018c15f4525bc42c7ffd74e98e -> d62cbf84ddeab29b7c3c78c09a7e7d27debc68018c15f4525bc42c7ffd74e98e: 25
--------------------------------------------------
Block 2:
Hash: 00d9d882ba762502523e628cb163e32a6af7ec17dfd7171d123c1e9edbbd1520
Previous Hash: 0034cf1d48a0ad1a32eca65314f3a8609c67666e0ebc516316d9c1f4a0ac7687
Nonce: 97
Transactions:
  None -> d62cbf84ddea